In [1]:
# 1️⃣ VERIFICACIÓN RÁPIDA DEL SISTEMA
import sys
import os
import json
import importlib

print("🔍 VERIFICACIÓN INICIAL")
print("=" * 40)

# Verificar dependencias críticas
deps_ok = True
for dep in ['dash', 'plotly', 'pandas', 'numpy']:
    try:
        __import__(dep)
        print(f"✅ {dep}")
    except ImportError:
        print(f"❌ {dep} - INSTALAR: pip install {dep}")
        deps_ok = False

# Verificar archivo de datos
json_path = 'Results/modelos_completos_por_celda.json'
if os.path.exists(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Extraer variables necesarias para la aplicación
    if isinstance(data, dict) and 'modelos_por_celda' in data:
        modelos_por_celda = data['modelos_por_celda']
        detalles_por_celda = data.get('detalles_por_celda', {})
        print(f"✅ Estructura completa: modelos_por_celda y detalles_por_celda")
    else:
        # Si data es directamente modelos_por_celda (formato anterior)
        modelos_por_celda = data
        detalles_por_celda = {}
        print(f"✅ Formato directo: modelos_por_celda")
    
    total_modelos = sum(len(modelos) for modelos in modelos_por_celda.values())
    print(f"✅ Datos cargados: {total_modelos} modelos en {len(modelos_por_celda)} celdas")
else:
    print(f"❌ Archivo no encontrado: {json_path}")
    deps_ok = False

# Resultado final
if deps_ok:
    print("\n🎯 SISTEMA LISTO")
else:
    print("\n⚠️ CORREGIR ERRORES ANTES DE CONTINUAR")

🔍 VERIFICACIÓN INICIAL
✅ dash
✅ plotly
✅ pandas
✅ numpy
✅ Estructura completa: modelos_por_celda y detalles_por_celda
✅ Datos cargados: 174 modelos en 6 celdas

🎯 SISTEMA LISTO


In [2]:
# 2️⃣ RECARGA DE MÓDULOS Y CONFIGURACIÓN
print("🔄 PREPARANDO MÓDULOS")
print("=" * 40)

# Recargar módulos críticos para asegurar código actualizado
modules = [
    'Modulos.Analisis_modelos.main_visualizacion_modelos',
    'Modulos.Analisis_modelos.plot_interactive',
    'Modulos.Analisis_modelos.ui_components',
    'Modulos.Analisis_modelos.data_loader'
]

for module_name in modules:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
        print(f"✅ {module_name.split('.')[-1]}")

# Configurar variables de entorno para debug
os.environ['DASH_DEBUG_CLICK'] = 'true'

# Verificar que los módulos se pueden importar
try:
    from Modulos.Analisis_modelos.main_visualizacion_modelos import main_visualizacion_modelos
    from Modulos.Analisis_modelos.plot_interactive import create_interactive_plot
    print("✅ Importaciones exitosas")
except Exception as e:
    print(f"❌ Error de importación: {e}")
    # Intentar ruta alternativa
    try:
        import sys
        sys.path.append('Modulos')
        from Analisis_modelos.main_visualizacion_modelos import main_visualizacion_modelos
        from Analisis_modelos.plot_interactive import create_interactive_plot
        print("✅ Importaciones exitosas (ruta alternativa)")
    except Exception as e2:
        print(f"❌ Error con ruta alternativa: {e2}")

print("\n🎯 MÓDULOS LISTOS")

🔄 PREPARANDO MÓDULOS
✅ Importaciones exitosas

🎯 MÓDULOS LISTOS


In [3]:
# 3️⃣ TEST CRÍTICO DE FUNCIONALIDAD
print("🔬 VERIFICANDO FUNCIONALIDAD DE CLICK")
print("=" * 40)

try:
    # Test crítico: ¿se genera customdata correctamente?
    if 'data' in globals() and data:
        print(f"🔍 Estructura de datos detectada:")
        print(f"   Claves principales: {list(data.keys())}")
        
        # Detectar estructura de datos
        if 'modelos_por_celda' in data and 'detalles_por_celda' in data:
            # Estructura con modelos_por_celda y detalles_por_celda
            modelos_data = data['modelos_por_celda']
            detalles_data = data['detalles_por_celda']
            
            if modelos_data:
                # Obtener una celda de ejemplo
                celda_ejemplo = list(modelos_data.keys())[0]
                print(f"   Celda ejemplo: {celda_ejemplo}")
                
                # Intentar extraer aeronave y parametro
                if '|' in celda_ejemplo:
                    aeronave, parametro = celda_ejemplo.split('|', 1)
                else:
                    # Usar valores por defecto si no hay separador
                    aeronave = "TestAeronave"
                    parametro = "TestParametro"
                
                modelos_test = modelos_data[celda_ejemplo][:2]
                modelos_filtrados = {celda_ejemplo: modelos_test}
                
                # Llamar a la función con la firma correcta
                fig_test = create_interactive_plot(
                    modelos_filtrados=modelos_filtrados,
                    aeronave=aeronave,
                    parametro=parametro,
                    show_training_points=True,
                    show_model_curves=True,
                    detalles_por_celda=detalles_data
                )
                
                # Verificar customdata
                has_customdata = False
                for i, trace in enumerate(fig_test.data):
                    if hasattr(trace, 'customdata') and trace.customdata is not None:
                        has_customdata = True
                        print(f"✅ CustomData encontrado en traza {i}")
                        break
                
                if has_customdata:
                    print("✅ CustomData generado correctamente")
                else:
                    print("❌ PROBLEMA: CustomData faltante")
                    
                print(f"✅ Figura test: {len(fig_test.data)} trazas")
                print(f"✅ Test con aeronave: {aeronave}, parámetro: {parametro}")
            else:
                print("❌ modelos_por_celda está vacío")
        else:
            print("❌ Estructura de datos no reconocida")
        
    else:
        print("❌ Datos no disponibles - ejecutar celda 1")
        
except Exception as e:
    print(f"❌ Error en test: {e}")
    # Debug adicional
    import traceback
    traceback.print_exc()
    
    # Test alternativo más simple
    try:
        print("🔄 Test alternativo...")
        if 'data' in globals():
            print(f"✅ Datos disponibles: {len(data)} claves principales")
            if isinstance(data, dict):
                for key, value in data.items():
                    print(f"   {key}: {type(value)} - {len(value) if hasattr(value, '__len__') else 'N/A'}")
        else:
            print("❌ Variable 'data' no encontrada")
    except Exception as e2:
        print(f"❌ Error en test alternativo: {e2}")

print("\n🎯 TEST COMPLETADO")

INFO:Modulos.Analisis_modelos.plot_model_curves:Extraídos 2 puntos imputados de detalles para celda A7|Payload
INFO:Modulos.Analisis_modelos.plot_model_curves:Filtrados 2 puntos imputados por métodos: ['final', 'similitud', 'correlacion']


🔬 VERIFICANDO FUNCIONALIDAD DE CLICK
🔍 Estructura de datos detectada:
   Claves principales: ['modelos_por_celda', 'detalles_por_celda']
   Celda ejemplo: A7|Payload
✅ CustomData encontrado en traza 0
✅ CustomData generado correctamente
✅ Figura test: 8 trazas
✅ Test con aeronave: A7, parámetro: Payload

🎯 TEST COMPLETADO


In [4]:
# 4️⃣ LANZAR APLICACIÓN - CELDA COMPLETA Y AUTOSUFICIENTE
# Esta celda incluye TODO lo necesario: datos, módulos, importaciones y opciones

import sys
import os
import json
import importlib
import psutil

print("🚀 INICIANDO APLICACIÓN DE ANÁLISIS - CELDA COMPLETA")
print("=" * 60)

# 1. VERIFICACIÓN Y CARGA DE DATOS
print("1️⃣ VERIFICANDO Y CARGANDO DATOS...")
try:
    # Si las variables ya existen, las usamos
    if 'modelos_por_celda' in globals() and modelos_por_celda:
        print(f"✅ Variables ya disponibles: {len(modelos_por_celda)} celdas")
        if 'detalles_por_celda' not in globals():
            detalles_por_celda = {}
    else:
        # Cargar desde archivo
        json_path = 'Results/modelos_completos_por_celda.json'
        if os.path.exists(json_path):
            with open(json_path, 'r') as f:
                data = json.load(f)
            
            if isinstance(data, dict) and 'modelos_por_celda' in data:
                modelos_por_celda = data['modelos_por_celda']
                detalles_por_celda = data.get('detalles_por_celda', {})
                print(f"✅ Estructura completa cargada")
            else:
                modelos_por_celda = data
                detalles_por_celda = {}
                print(f"✅ Formato directo cargado")
        else:
            raise FileNotFoundError(f"Archivo no encontrado: {json_path}")
    
    total_modelos = sum(len(modelos) for modelos in modelos_por_celda.values())
    print(f"✅ Total disponible: {total_modelos} modelos en {len(modelos_por_celda)} celdas")
    
except Exception as e:
    print(f"❌ Error cargando datos: {e}")
    print("💡 Asegúrate de que existe Results/modelos_completos_por_celda.json")
    raise

# 2. RECARGA DE MÓDULOS
print("\n2️⃣ RECARGANDO MÓDULOS...")
modules_to_reload = [
    'Modulos.Analisis_modelos.main_visualizacion_modelos',
    'Modulos.Analisis_modelos.plot_interactive',
    'Modulos.Analisis_modelos.ui_components',
    'Modulos.Analisis_modelos.data_loader'
]

for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
        print(f"✅ {module_name.split('.')[-1]} recargado")

# 3. IMPORTAR FUNCIÓN PRINCIPAL
print("\n3️⃣ IMPORTANDO FUNCIÓN PRINCIPAL...")
try:
    from Modulos.Analisis_modelos.main_visualizacion_modelos import main_visualizacion_modelos
    print("✅ Función main_visualizacion_modelos importada")
except Exception as e:
    print(f"❌ Error importando: {e}")
    # Intentar ruta alternativa
    try:
        sys.path.append('Modulos')
        from Analisis_modelos.main_visualizacion_modelos import main_visualizacion_modelos
        print("✅ Función importada (ruta alternativa)")
    except Exception as e2:
        print(f"❌ Error con ruta alternativa: {e2}")
        raise

# 4. CONFIGURACIONES
print("\n4️⃣ CONFIGURANDO MODOS...")
config_debug = {
    'json_path': 'Results/modelos_completos_por_celda.json',
    'use_dash': True,
    'port': 8054,
    'debug': True
}

config_produccion = {
    'json_path': 'Results/modelos_completos_por_celda.json',
    'use_dash': True,
    'port': 8055,
    'debug': False
}

print("📋 OPCIONES DISPONIBLES:")
print("   🐛 MODO DEBUG    (Puerto 8054) - Con mensajes detallados")
print("   🚀 MODO PRODUCCIÓN (Puerto 8055) - Interfaz limpia")

print("\n🌐 ENLACES DIRECTOS:")
print(f"   🐛 DEBUG:      http://localhost:{config_debug['port']}")
print(f"   🚀 PRODUCCIÓN: http://localhost:{config_produccion['port']}")

# 5. VERIFICAR PUERTOS
print("\n5️⃣ VERIFICANDO PUERTOS...")
puertos_activos = []
for port in [8054, 8055]:
    puerto_ocupado = False
    for conn in psutil.net_connections():
        if conn.laddr.port == port and conn.status == 'LISTEN':
            print(f"⚠️  Puerto {port} ya está en uso (PID: {conn.pid})")
            puertos_activos.append(port)
            puerto_ocupado = True
            break
    if not puerto_ocupado:
        print(f"✅ Puerto {port} disponible")

# 6. OPCIONES DE LANZAMIENTO
print("\n" + "=" * 60)
print("🎯 ELIGE UNA OPCIÓN (descomenta SOLO UNA línea):")
print("=" * 60)

# OPCIÓN 1: MODO DEBUG ⬇️ (Descomenta esta línea)
#main_visualizacion_modelos(**config_debug)

# OPCIÓN 2: MODO PRODUCCIÓN ⬇️ (Descomenta esta línea)
main_visualizacion_modelos(**config_produccion)

print("\n💡 INSTRUCCIONES:")
print("   1. Descomenta SOLO UNA de las líneas de arriba")
print("   2. Ejecuta esta celda nuevamente")
print("   3. La aplicación se abrirá automáticamente")
print("   4. Presiona Ctrl+C para detener")

print("\n🎯 DIFERENCIAS ENTRE MODOS:")
print("   🐛 DEBUG: Mensajes detallados, recarga automática, dev tools")
print("   🚀 PRODUCCIÓN: Interfaz limpia, mayor rendimiento, sin dev tools")
print("   ✅ AMBOS MODOS: Funcionalidad completa de click garantizada")

print("\n✅ ESTADO: Todo preparado y listo para lanzar")
print("🔧 INCLUYE: Datos ✓ Módulos ✓ Importaciones ✓ Configuraciones ✓")

INFO:Modulos.Analisis_modelos.main_visualizacion_modelos:Cargando datos desde: Results/modelos_completos_por_celda.json
INFO:Modulos.Analisis_modelos.data_loader:Cargados 6 celdas con modelos
INFO:Modulos.Analisis_modelos.data_loader:Cargados 6 celdas con detalles
INFO:Modulos.Analisis_modelos.main_visualizacion_modelos:Datos cargados exitosamente


🚀 INICIANDO APLICACIÓN DE ANÁLISIS - CELDA COMPLETA
1️⃣ VERIFICANDO Y CARGANDO DATOS...
✅ Variables ya disponibles: 6 celdas
✅ Total disponible: 174 modelos en 6 celdas

2️⃣ RECARGANDO MÓDULOS...
✅ main_visualizacion_modelos recargado
✅ plot_interactive recargado
✅ ui_components recargado
✅ data_loader recargado

3️⃣ IMPORTANDO FUNCIÓN PRINCIPAL...
✅ Función main_visualizacion_modelos importada

4️⃣ CONFIGURANDO MODOS...
📋 OPCIONES DISPONIBLES:
   🐛 MODO DEBUG    (Puerto 8054) - Con mensajes detallados
   🚀 MODO PRODUCCIÓN (Puerto 8055) - Interfaz limpia

🌐 ENLACES DIRECTOS:
   🐛 DEBUG:      http://localhost:8054
   🚀 PRODUCCIÓN: http://localhost:8055

5️⃣ VERIFICANDO PUERTOS...
✅ Puerto 8054 disponible
✅ Puerto 8055 disponible

🎯 ELIGE UNA OPCIÓN (descomenta SOLO UNA línea):
Iniciando aplicación Dash en http://localhost:8055
Presione Ctrl+C para detener la aplicación



💡 INSTRUCCIONES:
   1. Descomenta SOLO UNA de las líneas de arriba
   2. Ejecuta esta celda nuevamente
   3. La aplicación se abrirá automáticamente
   4. Presiona Ctrl+C para detener

🎯 DIFERENCIAS ENTRE MODOS:
   🐛 DEBUG: Mensajes detallados, recarga automática, dev tools
   🚀 PRODUCCIÓN: Interfaz limpia, mayor rendimiento, sin dev tools
   ✅ AMBOS MODOS: Funcionalidad completa de click garantizada

✅ ESTADO: Todo preparado y listo para lanzar
🔧 INCLUYE: Datos ✓ Módulos ✓ Importaciones ✓ Configuraciones ✓


In [5]:
# 5️⃣ DEBUG EN TIEMPO REAL (Solo si hay problemas)
import psutil
import requests
from datetime import datetime

print("🔧 DIAGNÓSTICO RÁPIDO")
print("=" * 40)

# 1. Estado de puertos
print("📡 PUERTOS ACTIVOS:")
puertos_dash = [8050, 8051, 8052, 8053, 8054]
activos = []

for port in puertos_dash:
    for conn in psutil.net_connections():
        if conn.laddr.port == port and conn.status == 'LISTEN':
            activos.append(port)
            print(f"   ✅ {port} (PID: {conn.pid})")
            break
    else:
        print(f"   ❌ {port}")

# 2. Test de conectividad del puerto principal
if 8054 in activos:
    try:
        resp = requests.get("http://localhost:8054", timeout=2)
        print(f"✅ Puerto 8054 responde (HTTP {resp.status_code})")
    except:
        print("❌ Puerto 8054 no responde")

# 3. Variables críticas en memoria
critical_vars = ['data', 'modelos_por_celda', 'total_modelos']
print(f"\n🧠 VARIABLES CRÍTICAS:")
for var in critical_vars:
    if var in globals():
        if var == 'total_modelos':
            print(f"   ✅ {var}: {globals()[var]}")
        else:
            print(f"   ✅ {var}: disponible")
    else:
        print(f"   ❌ {var}: faltante")

print(f"\n⏰ Diagnóstico: {datetime.now().strftime('%H:%M:%S')}")

🔧 DIAGNÓSTICO RÁPIDO
📡 PUERTOS ACTIVOS:
   ❌ 8050
   ❌ 8051
   ❌ 8052
   ❌ 8053
   ❌ 8054

🧠 VARIABLES CRÍTICAS:
   ✅ data: disponible
   ✅ modelos_por_celda: disponible
   ✅ total_modelos: 174

⏰ Diagnóstico: 23:48:52
